In [1]:
import pandas as pd
import numpy as np
import os, time, requests, json

In [5]:
# Add FDR, Game week number

input_paths            = [f"./data/20{i}-{i+1}/" for i in range(18, 24)]
for dir in input_paths:
    fixtures           = pd.read_csv(os.path.join(dir, 'fixtures.csv'))
    
    # Note: These geniuses switched up home and away
    fixtures           = fixtures[fixtures['minutes'] > 0]
    fixtures.dropna(subset = ['event'], inplace = True)
    home_diff          = fixtures.set_index('id')['team_a_difficulty'] 
    away_diff          = fixtures.set_index('id')['team_h_difficulty']
    game_week          = fixtures.set_index('id')['event']

    for player in os.listdir(dir + "/players/"):
        if player[0] == '.':
            continue
        df             = pd.read_csv(os.path.join(dir + "/players/", player, 'gw.csv'))
        if not {'own_FDR', 'opp_FDR'}.issubset(df.columns):
            df['own_FDR']  = np.where(df['was_home'], df['fixture'].map(home_diff), df['fixture'].map(away_diff))
            df['opp_FDR']  = np.where(df['was_home'], df['fixture'].map(away_diff), df['fixture'].map(home_diff))
        if 'gw' not in df.columns:
            df['gw']       = df['fixture'].map(game_week).map(int)
        df.to_csv(os.path.join(dir + "/players/", player, 'gw.csv'))